In [1]:
import designMethods.en_13001_3_3 as en
from output import create_output_file
from IPython.display import display, FileLink
import ipywidgets as widgets
import pathlib
import os
import itertools

In [2]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix

In [3]:
en_13001 = {}
en_13001["input"] = en.EN_input()
en_13001["computed"] = en.Computed_data()
gen_vars = {"num_run": 1, "input_file": None, "output_file": None}
direction = 1

In [4]:
def read_file(input_file):
    en_13001["input"].load_gp_input(input_file, "configuration")

    # load material and en 13001 parameters
    en_13001["input"].load_material_input_check(input_file, "rail_materials", "wheel_materials")
    en_13001["input"].load_parameter_input(input_file, "Input_variables")

    # check for input errors and drop faulty configurations
    en_13001["input"].perform_error_checks()
    en_13001["input"].drop_error_configs()

    # precompute factors and material parameters
    en_13001["input"].parameters.compute_f_f3()
    en_13001["input"].parameters.compute_contact_and_f_1()
    en_13001["input"].set_materials_and_geometry()

In [5]:
def init_gps(change):
    msg_box.value = "Initializing GPs: this may take up to 30 seconds"
    en_13001["input"].config = drop_config.value
    gp_configs = {"m1": {1: "m1l", -1: "m1r"}, "m2": {1: "m2"}}
    gp_config = gp_configs[en_13001["input"].config][direction]
    parts = ["wf", "wr", "r"]
    en_13001["computed"].get_gps_kc(gp_config, parts)
    en_13001["input"].config_loaded = True
    msg_box.value = "Initialized GPs"

In [6]:
def start_computation(change):
    if en_13001["input"].parameters.loaded is None:
        if uploader._counter == 0:
            msg_box.value = "Please upload an excel file"
            return
    
    if uploader._counter > 0:
        gen_vars["input_file"] = "input.xlsx"
        with open(gen_vars["input_file"], "w+b") as i:
            i.write(uploader.data[-1])
        read_file(gen_vars["input_file"])
        os.remove(gen_vars["input_file"])
        uploader.value.clear()
        uploader._counter = 0
        
        if en_13001["input"].config_loaded is None:
            init_gps(None)
        msg_box.value = "Start Computation"
        en_13001["input"].recompute_gp_data(en_13001["input"].config)
        en_13001["computed"].predict_kc(en_13001["input"].gp_input.norm)
        en_13001["computed"].compute_F_sd_f_all(en_13001["input"].gp_input.raw, en_13001["input"].config, direction)
        en_13001["computed"].predict_travelled_dist(en_13001["input"].parameters.data)

        # create computation instance and compute configs
        en_13001["computation"] = en.Computation(en_13001["input"], en_13001["computed"])
        en_13001["computation"].compute_pre_F_rd_all()
        en_13001["computation"].compute_F_rd_all()
        en_13001["computation"].compute_proofs_all()

        # results output
        en_13001["input"].prepare_for_output()
        en_13001["computation"].load_results_all()
    msg_box.value = "Finished computation"
    gen_vars["output_file"] = "./output.xlsx"
    create_output_file(en_13001["computation"], en_13001["input"], gen_vars["output_file"])
    local_file = FileLink(gen_vars["output_file"], result_html_prefix="Click here to download results: ")
    out_file.clear_output()
    out_errors.clear_output()
    with out_file:
        display(local_file)
        display(btn_delete_output)
    with out_errors:
        for error in list(itertools.chain(*en_13001["input"].error_report)):
            print(error)

In [7]:
def delete_output_file(change):
    os.remove(gen_vars["output_file"])
    out_file.clear_output()

In [8]:
uploader = widgets.FileUpload(accept=".xlsx", multiple=False)
drop_mode = widgets.Dropdown(description="Computation Mode", options=["proof", "min_diameter"], value="proof")
drop_config = widgets.Dropdown(description="SC Configuration", options=["m1", "m2"], value="m1")
btn_update_config = widgets.Button(description="Update Configuration")
btn_update_config.on_click(init_gps)
btn_start = widgets.Button(description="Start computation")
btn_start.on_click(start_computation)
out_file = widgets.Output()
out_errors = widgets.Output()
msg_box = widgets.Text(value="", description="Message Box")
btn_delete_output = widgets.Button(description="Delete Resultfile")
btn_delete_output.on_click(delete_output_file)

In [9]:
widgets.VBox([widgets.Label("MARS Demonstrator"),
              drop_mode, drop_config, btn_update_config, uploader, msg_box, btn_start, out_file, out_errors])